In [1]:
from credoai.utils.credo_api_utils import *

In [3]:
scope = get_solution_manifest('ZESsXtfyDRWzftnRZt3QZS')
scope['metrics']

[{'id': 'A7JCD7y3RjCVp9v7Q45jpB',
  'type': 'ai_scope_metrics',
  'lower_threshold': 0.0,
  'model_id': 'Yppp7ETdViNFfDJXUSpBLE',
  'name': 'recall score',
  'upper_threshold': 1.0},
 {'id': 'e8ZXC3D5WKCsVG3nPW2cki',
  'type': 'ai_scope_metrics',
  'lower_threshold': -0.1,
  'model_id': 'Yppp7ETdViNFfDJXUSpBLE',
  'name': 'equalized odds',
  'upper_threshold': 0.1},
 {'id': 'Hz4bUeseP7vvqwKWhkcA9A',
  'type': 'ai_scope_metrics',
  'lower_threshold': 0.0,
  'model_id': 'Yppp7ETdViNFfDJXUSpBLE',
  'name': 'precision_score',
  'upper_threshold': 1.0},
 {'id': 'h8pQbEWxj8EySG2hqNbLKm',
  'type': 'ai_scope_metrics',
  'lower_threshold': 0.0,
  'model_id': 'Vw2dUofxXJan3NBcECWnLk',
  'name': 'equal odds',
  'upper_threshold': 1.0},
 {'id': 'N4LSTVjeXqFVFhdmMNS7iX',
  'type': 'ai_scope_metrics',
  'lower_threshold': 0.0,
  'model_id': 'Yppp7ETdViNFfDJXUSpBLE',
  'name': 'tpr',
  'upper_threshold': 1.0}]

# Customizing Lens

Lens 

## Parameterizing Assessments

## Creating New Modules & Assessments

## Custom Metrics

### incorporating confidence intervals

Custom metrics can be incorporated by creating a `FairnessFunction`. The `FairnessFunction` is a lightweight wrapper class that defines a few characteristics of the custom function needed by the module. 

**Example: Confidence Intervals**

Confidence intervals are not generically supported. However, they can be derived for metrics derived from a confusion matrix using the `wilson confidence interval`. A convenience function called `confusion_wilson` is supplied which returns an array: [lower, upper] bound for the metric. 

However, the module is not designed to accomodate metrics that return arrays. If you wish to incorporate CIs, we'd suggest creating a derived function that returns the lower bound, for example, and then wrapping it in a `FairnessFunction` as we mentioned for custom functions above. We show a case of that below:

In [ ]:
from credoai.utils.metrics import confusion_wilson
from credoai.assessment.model_assessments.fairness_base import FairnessFunction

# define partial functions for the true positive rate lower bound
def lower_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='tpr', confidence=0.95)[0]

# and upper bound
def upper_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='tpr', confidence=0.95)[1]

# wrap the functions in fairness functions
lower_metric = FairnessFunction(name = 'lower_bound_tpr', 
                          func = lower_bound_tpr, 
                          takes_sensitive_attributes = False, 
                          takes_prob = False)

upper_metric = FairnessFunction(name = 'upper_bound_tpr', 
                          func = upper_bound_tpr, 
                          takes_sensitive_attributes = False, 
                          takes_prob = False)
           
# then proceed as normally
custom_assessment = FairnessModule([lower_metric, 
                           'true_positive_rate', 
                           upper_metric],
                A_str_test,
                Y_test,
                test_preds,
                test_scores,
                fairness_bounds
               )

custom_assessment.get_disaggregated_results(calculate_risk=False)